In [ ]:
import uproot

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys

# import plothist
# plt.style.use("belle2")
# from plothist import make_2d_hist, plot_2d_hist_with_projections

In [ ]:
sys.path.append("/home/belle2/amubarak/Ds2D0enue_Analysis/07-Python_Functions/")

# Prep-Work

### Import Data

In [ ]:
# In this notebook we only process the main signal and the generic events,
# for illustration purposes.
# You can add other backgrounds after if you wish.
samples1 = ["ccbar","All"]
samples2 = ["ccbar_SideBand","All_SideBand"]

In [ ]:
DataFrames = {}  # define empty dictionary to hold dataframes
Date = "0327"
Attempt = "0"
Code = "Sideband"

# Signal:
DataFrames["Signal"] =  uproot.concatenate("/home/belle2/amubarak/C01-Simulated_Events/Ds2D0enu-Signal.root:D02kmpippi0tree",library='pd')
# Background
for s in samples1[0:]: # loop over samples
    DataFrames[s] =  uproot.concatenate("/group/belle2/users2022/amubarak/02-Grid/Completed/Ds2D0e-Generic_Ds_" + Date +"25_"+ Attempt +"_"+ s +".root:D02kmpippi0tree",library='pd')

for s in samples2[0:]: # loop over samples
    DataFrames[s] =  uproot.concatenate("/group/belle2/users2022/amubarak/02-Grid/Completed/Ds2D0e-Generic_Ds_" + Date +"25_"+ Code +"_"+ s +".root:D02kmpippi0tree",library='pd')

In [ ]:
pd.set_option('display.max_rows', 200000)
pd.set_option('display.max_columns', 200000)

The line below is to look at the available variables.

In [ ]:
DataFrames["Signal"].columns.tolist()

In [ ]:
DataFrames["All"].isna().sum()

### Setup
The code below will be used to apply cuts to the data.  
The range of the plots.

In [ ]:
# Electron ID
#-------------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['e_electronID']>=0.95]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['e_electronID']>=0.95]
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_gammaveto_em_electronID']>=0.95]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_gammaveto_em_electronID']>=0.95]

# Photon Conversion
#-------------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=0.1]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_gammaveto_M_Correction']>=0.1]

# Peaking Background Removal
#----------------------------
# DataFrames["All"] = DataFrames["All"][(DataFrames["All"]['Ds_diff_D0pi']>=0.15)]
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_diff_D0pi']>=0.15)]

# # Vertex Fitting
# #----------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_chiProb']>=0.01]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_chiProb']>=0.01]

# Dalitz Removal
#----------------------------
# DataFrames["All"] = DataFrames["All"][(DataFrames["All"]['Ds_pi0veto_M_Correction']<=0.08) | (DataFrames["All"]['Ds_pi0veto_M_Correction']>=0.16)]
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_pi0veto_M_Correction']<=0.08) | (DataFrames["Signal"]['Ds_pi0veto_M_Correction']>=0.16)]

# Vertex Fit
#----------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_chiProb_rank']==1]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_chiProb_rank']==1]

# D0 Invariant Mass
#-----------------------
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_D0_sideband']==1)]
# DataFrames["All"] = DataFrames["All"][(DataFrames["All"]['Ds_D0_sideband']==1)]

# KaonID
#-----------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['K_kaonID']>=0.5]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['K_kaonID']>=0.5]

In [ ]:
features = DataFrames["All"].columns.tolist()
features.remove('Ds_goodDsplus')

In [ ]:
# # Suffix removal from String list
# # using list comprehension + endswith()
# features = [ele for ele in features if not ele.endswith('charge')
#             and not ele.endswith('trackTime')
#             and not ele.endswith('isSignal') and not ele.endswith('PDG')
#             and not ele.endswith('mcErrors')
#             and not ele.endswith('genMotherPDG') and not ele.endswith('genMotherPDG_1') and not ele.endswith('genMotherPDG_2')
#             and not ele.endswith('genMotherID') and not ele.endswith('genMotherID_1') and not ele.endswith('genMotherID_2')
#             and not ele.endswith('genParticleID')
#             and not ele.endswith('nMCDaughters') and not ele.endswith('nMCMatches') and not ele.endswith('mcMatchWeight')
#             and not ele.endswith('mcInitial') and not ele.endswith('mcVirtual')
#             and not ele.endswith('mcSecPhysProc') and not ele.endswith('mcPrimary') and not ele.endswith('mcPhotos') 
#             and not ele.endswith('isMisidentified') and not ele.endswith('isCloneTrack') and not ele.endswith('isOrHasCloneTrack')
#             and not ele.endswith('mcP') and not ele.endswith('mcE')
#             and not ele.endswith('seenInCDC') and not ele.endswith('seenInPXD') and not ele.endswith('seenInSVD')
#             ]

# # Initializing remove list
# remove_list = ['__experiment__','__run__','__event__','__production__','__candidate__','__ncandidates__','__weight__','__eventType__',
#                'e_pminusptrue_p','e_pminusptrue_old','e_mcMother_E','e_mcMother_p','e_mcMother_mcDecayVertexFromIPDistance',
#                'e_Motherptrueminusp_p','e_Motherptrueminusp_E',
#                'e_M','K_M','pi_M',
#                'D0_mcMother_mcDaughter_1_pt','D0_mcMother_mcDaughter_1_pz','D0_mcMother_mcDaughter_1_cos_theta','D0_ifNANgiveX_isSignal_5',
#                'Ds_M_pi','Ds_M_uncorrected','Ds_daughterCombination_M_0_0_1',
#                'Ds_decayAngle_0','Ds_cos_decayAngle_0','Ds_decayAngle_1','Ds_cos_decayAngle_1',
#                'Ds_openangle','Ds_Angle_D0e',
#                'Ds_genNStepsToDaughter_0','Ds_genNStepsToDaughter_1','Ds_genNMissingDaughter_11','Ds_genNMissingDaughter_22',
#                'Ds_ifNANgiveX_isSignal_5',
#                'Ds_Comb','Ds_Failed','Ds_Dstar0','Ds_Dstarplus','Ds_Other','Ds_Signal',
#                'Ds_gammaveto_em_GoodElectron',
#                'Ds_D0_Dstarplus','Ds_D0_Dstar0','Ds_D0_NoDstarplusDstar0','Ds_D0_Other',
#                'Ds_azimuthalAngleInDecayPlane_0_1',
#                'Ds_daughterMotherDiffOf_0_vertexDistance',
#                'Ds_gammaveto_ep_M','Ds_gammaveto_em_M','Ds_Dstar0Mode1_chiProb','Ds_Dstar0Mode2_chiProb',
#                'Ds_D0_sideband',
#                'IPX','IPY','IPZ']
 
# # Removing elements present in other list
# # using list comprehension
# features = [i for i in features if i not in remove_list]

In [ ]:
def get_histogram_range(data, iqr_factor=1.5, lower_percentile=1, upper_percentile=99, padding=0.05):
    """
    Compute a histogram range while excluding outliers.

    Parameters:
        data (pd.Series or np.array): Input data for range calculation.
        iqr_factor (float): Multiplier for IQR-based outlier removal (default=1.5).
        lower_percentile (float): Minimum percentile cutoff (default=1).
        upper_percentile (float): Maximum percentile cutoff (default=99).
        padding (float): Fraction of range to expand for better visualization (default=5%).

    Returns:
        (float, float): Robust range (min, max) for the histogram.
    """
    data = np.array(data, dtype=np.float64)  # Ensure data is numeric

    # Remove NaNs and infinite values
    data = data[np.isfinite(data)]
    
    if len(data) == 0:
        return (0, 1)  # Fallback for empty data

    if len(np.unique(data)) == 1:
        return (data[0] - 0.1, data[0] + 0.1)  # Single-value case

    # Compute quartiles
    q1, q3 = np.percentile(data, [25, 75])
    iqr = q3 - q1

    # Define IQR-based outlier boundaries
    lower_bound = q1 - iqr_factor * iqr
    upper_bound = q3 + iqr_factor * iqr

    # Further refine using percentiles
    min_cutoff = np.percentile(data, lower_percentile)
    max_cutoff = np.percentile(data, upper_percentile)

    lower_bound = max(lower_bound, min_cutoff)
    upper_bound = min(upper_bound, max_cutoff)

    # Ensure padding for better visualization
    range_span = upper_bound - lower_bound
    if range_span == 0:  # If no spread, widen artificially
        return (lower_bound - 0.1, upper_bound + 0.1)

    lower_bound -= range_span * padding
    upper_bound += range_span * padding

    return (lower_bound, upper_bound)

# Single Variable  
The code below is to look at any specific variable closely.

Below, I look for good BDT variables

In [ ]:
Stacked = False
Density = True
Bins = 50
Compared = True

Op = -1
dM = -1
Hits = -1

for i in features:
    label1= r'$isSignal(D^{0}) \neq 1$'
    label2= r'$isSignal(D^{0})=1$'

    labels=[label1,label2]
    colors=['purple',"#D55E00"]
    data=[
            DataFrames["All"][(abs(DataFrames["All"]['D0_isSignal'])==0) | (DataFrames["All"]['D0_isSignal'].isna())][i],
            DataFrames["All"][(abs(DataFrames["All"]['D0_isSignal'])==1)][i]
    ]

    Range = get_histogram_range(DataFrames['All'][i])
    # Range = [DataFrames['Signal'][i].min(axis=0),DataFrames['Signal'][i].max(axis=0)]

    factor = 0.25
    plt.hist(DataFrames["Signal"][i], label="Signal", color='#7eb0d5',density=Density, bins=Bins, alpha=0.8, range=Range)
    plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)

    plt.ylabel('Entries')
    plt.xlabel(i)
    plt.legend()
    plt.show()

Below, I check that the sideband is truly usable

In [ ]:
Stacked = False
Density = True
Bins = 50
Compared = True

Op = -1
dM = -1
Hits = -1

for i in features:
    label1= r'$isSignal(D^{0}) \neq 1$'
    label2= r'$Sideband$'

    labels=[label1,label2]
    colors=['purple',"#D55E00"]
    data=[
            DataFrames["All"][(abs(DataFrames["All"]['D0_isSignal'])==0) | (DataFrames["All"]['D0_isSignal'].isna())][i],
            DataFrames["All_SideBand"][i]
    ]

    Range = get_histogram_range(DataFrames['All'][i])
    # Range = [DataFrames['Signal'][i].min(axis=0),DataFrames['Signal'][i].max(axis=0)]

    factor = 0.25
    plt.hist(DataFrames["Signal"][i], label="Signal", color='#7eb0d5',density=Density, bins=Bins, alpha=0.8, range=Range)
    plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)

    plt.ylabel('Entries')
    plt.xlabel(i)
    plt.legend()
    plt.show()